# Notebook 17: Collaborative Filtering
***

During lecture, we discussed Collaborative Filtering, a common method for making user recommendations and estimating unknown user ratings for new items they have not yet rated/tried. An important step in collaborative filtering is to determine the $k$ items in our data set most similar to a given item; this is the $k$-nearest neighbors problem. To solve it efficiently, we will introduce and practice using the `NearestNeighbors` method from the ubitquitous Scikit-learn (`sklearn`) package.

We'll need numpy, Pandas and the Scikit-learn NearestNeighbors packages for this notebook, so let's load them.

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

<br>

### Exercise 1: Meeting your neighbors

Suppose we have a data set consisting of four users and their ratings (0 = hate it, 10 = love it) for three candy bars as follows:

In [2]:
dfCandy = pd.DataFrame({"Jeanne" : [3, 5, 10],
                        "George" : [0, 9, 7],
                        "Kathy" : [5, 6, 1],
                        "Rich" : [9, 4, 4]})

# These are the candy bars corresponding to the rows of the data frame
treats = ["Whatchamacallit", "Milky Way", "Almond Joy"]
dfCandy.rename({0 : treats[0], 1 : treats[1], 2 : treats[2]}, inplace=True)

dfCandy.head()

,George,Jeanne,Kathy,Rich
Whatchamacallit,0,3,5,9
Milky Way,9,5,6,4
Almond Joy,7,10,1,4


Finish the following function to compute the distances from a given user in a DataFrame to each other user, and return the indices of the $k$ users with the lowest distances to the given user. You can either write your own function to compute Euclidean distance or use the `numpy.linalg.norm` function.

As a brief test, the distances from Jeanne to each of the users are: `[0, 5.83, 9.27, 8.54]`

In [3]:
def neighbo.lirs(df, user, k=1):
    '''
    Loop over all users and compute the distance to the input user.
    Return the indices/names of the k nearest other users as a list.
    '''
    distances = [0, 0, 0, 0] # TODO
    # TODO -- probably some other calculations
    return nearest

Suppose a new user, Elizabeth, is considering trying a Whatchamacallit bar, and has 

<br>

### Exercise 2: Item-item collaborative filtering

Our homemade code for finding the $k$ nearest neighbors from **Exercise 1** is good and all, but - as with most things - there are some nice Python packages that can also take care of this in a more efficient manner. The homemade approach will become less and less efficient as the size of our utility matrix grows, so let's explore using Scikit-learn's [`NearestNeighbors`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) methods.

Suppose we have a set of movies A-F and 12 users given by the utility matrix below (and from lecture).

In [12]:
movies = [[1,0,3,0,0,5,0,0,5,0,4,0],
          [0,0,5,4,0,0,4,0,0,2,1,3],
          [2,4,0,1,2,0,3,0,4,3,5,0],
          [0,2,4,0,5,0,0,4,0,0,2,0],
          [0,0,4,3,4,2,0,0,0,0,2,5],
          [1,0,3,0,3,0,0,2,0,0,4,0]]

dfM = pd.DataFrame(movies)
dfM.rename({0:"A",1:"B",2:"C",3:"D",4:"E",5:"F"}, inplace=True)
dfM.head(6)


,0,1,2,3,4,5,6,7,8,9,10,11
A,1,0,3,0,0,5,0,0,5,0,4,0
B,0,0,5,4,0,0,4,0,0,2,1,3
C,2,4,0,1,2,0,3,0,4,3,5,0
D,0,2,4,0,5,0,0,4,0,0,2,0
E,0,0,4,3,4,2,0,0,0,0,2,5
F,1,0,3,0,3,0,0,2,0,0,4,0


We can start by setting up a `NearestNeighbors` object to find the two nearest neighbors to a given data point.

In [22]:
# Set up a NearestNeighbors object to find the two nearest neighbors
neigh = NearestNeighbors(n_neighbors=2)

Then we need to fit the model to the movies data set. Without suppressing the output, we can also see some of the specifics of the NearestNeighbors object that was fit.

In [25]:
# Fit the model based on the movies data
neigh.fit(movies)  
#note: fitting off movies *array* instead of df here, because by default kNN will exclude the point from
#its own nearest neighbors, as we might want in the next calculation


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

We can have a quick check on if our method is working properly by checking what the nearest neighbors of Movie A are. The first array in the output contains the distances to the neighbors, and the second array provides the indices within the data set of the nearest points. If we ask for the nearest points to a movie that's actually in the data set, what should the closest neighbor always be? Let's check!

In [ ]:
# What movie is most similar to movie A?
#print(dfM.loc["A",:])
print(neigh.kneighbors(np.reshape(dfM.loc["A",:], (1,-1))))  

#print(neigh.kneighbors(dfM.loc["A",:]))


Now that was using all of the data. Modify the code above to only use the movies also rated by User 5 for fitting the `NearestNeighbors` model. The `loc` method for Pandas DataFrames will be userful here.

In [ ]:
# Set up a NearestNeighbors object to find the two nearest neighbors
neigh = NearestNeighbors(n_neighbors=2)

# Fit the model based on the movies data using only data rated by user 5
neigh.fit()##TODO

# Now which of those movies is most similar to movie A?
print(neigh.kneighbors(np.reshape(dfM.loc["A",:], (1,-1)))) 

In the previous steps, you may have noticed that the distances computed are "[minkowski](https://en.wikipedia.org/wiki/Minkowski_distance)", which can correspond to either Manhattan distance or Euclidean distance. 

In [29]:
print(neigh)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)


Neither of these are what we used in lecture, which is the cosine distance. Check out the [Scikit-learn documentation on `NearestNeighbors`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) and make a modification to use the cosine distance instead. Note that the only algorithm option that works with the cosine distance is `algorithm="brute"`.

In [ ]:
# SOLUTION:

# Set up a NearestNeighbors object to find the two nearest neighbors
neigh = NearestNeighbors(#TODO)
# Fit the model based on the movies data
neigh.fit( #TODO)

# What movie is most similar to movie A?
print(neigh.kneighbors(np.reshape(dfM.loc["A",:], (1,-1))))

**Question:** It's close, but why doesn't this match what we saw in class? Perform a calculation that accounts for the difference and recompute the two nearest neighbors to Movie A and their distances. After that, they should match what we saw in lecture, and the world will be complete.

In [2]:
# SOLUTION: match input to kNN to that from lecture....

In [ ]:
# SOLUTION:

# Set up a NearestNeighbors object to find the two nearest neighbors
neigh = #TODO

# Fit the model based on the movies data
neigh.fit(#TODO)
# What movie is most similar to movie A?
print(neigh.kneighbors(np.reshape(dfM.loc["A",:], (1,-1))))